# Genshin Impact Knowledge Graph
# Submitted By: K16-Sarim-Raffay-Zaraar

In [1]:
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace, RDFS, OWL, FOAF
from rdflib.namespace import FOAF , XSD 
import urllib.parse
import numpy as np
import numpy as np

# Base Ontology

In [2]:
g = Graph()
g = g.parse('baseProject.rdf', format='application/rdf+xml')

gOnto = Namespace('http://www.genshin.org/ontology/')
gRes = Namespace('http://www.genshin.org/resource/')
foaf = Namespace("http://xmlns.com/foaf/0.1/")
wd = Namespace("http://www.wikidata.org/entity/")

g.bind('genshinOntology', gOnto)
g.bind('genshinResource', gRes)
g.bind('foaf', foaf)
g.bind('wd', wd)

print(g.serialize(format="turtle"))

@prefix : <http://www.genshin.org/> .
@prefix genshinOntology: <http://www.genshin.org/ontology/> .
@prefix genshinResource: <http://www.genshin.org/resource/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix swrl: <http://www.w3.org/2003/11/swrl#> .
@prefix swrla: <http://swrl.stanford.edu/ontologies/3.3/swrla.owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

swrla:isRuleEnabled a owl:AnnotationProperty .

genshinOntology: a owl:Ontology ;
    owl:versionIRI <http://www.genshin.org/ontology/1.0.0/> .

genshinOntology:AdvancedReaction a owl:Class ;
    rdfs:comment "Those specific reactions where more than 2 elements can participate in" ;
    rdfs:subClassOf genshinOntology:Reaction ;
    owl:equivalentClass [ a owl:Restriction ;
            owl:minQualifiedCardinality "3"^^xsd:nonNegativeInteger ;
            owl:onClass genshinOntology:Element ;
    

# Adding Reactions

In [3]:
df=pd.read_csv('./database/Reactions.csv',sep=",")
df.head()

,Element,causesReaction
0,Pyro,Overloaded
1,Electro,Overloaded
2,Pyro,Melt
3,Cryo,Melt
4,Pyro,Vaporize


In [4]:
for index, row in df.iterrows():
    g.add((URIRef(gRes+row['Element']), URIRef(gOnto+row.index[1]), URIRef(gRes+row['causesReaction'])))
    g.add((URIRef(gRes+row['Element']), RDFS.label, Literal(row['Element'], datatype = XSD.string)))
    g.add((URIRef(gRes+row['causesReaction']), RDFS.label, Literal(row['causesReaction'], datatype = XSD.string)))
    
print(g.serialize(format='turtle'))

@prefix : <http://www.genshin.org/> .
@prefix genshinOntology: <http://www.genshin.org/ontology/> .
@prefix genshinResource: <http://www.genshin.org/resource/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix swrl: <http://www.w3.org/2003/11/swrl#> .
@prefix swrla: <http://swrl.stanford.edu/ontologies/3.3/swrla.owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

swrla:isRuleEnabled a owl:AnnotationProperty .

genshinOntology: a owl:Ontology ;
    owl:versionIRI <http://www.genshin.org/ontology/1.0.0/> .

genshinOntology:AdvancedReaction a owl:Class ;
    rdfs:comment "Those specific reactions where more than 2 elements can participate in" ;
    rdfs:subClassOf genshinOntology:Reaction ;
    owl:equivalentClass [ a owl:Restriction ;
            owl:minQualifiedCardinality "3"^^xsd:nonNegativeInteger ;
            owl:onClass genshinOntology:Element ;
    

In [5]:
g.serialize(destination="afterAddingElements.rdf", format="application/rdf+xml")

<Graph identifier=Nf1cc6f9c6da0490896c863e1456403dc (<class 'rdflib.graph.Graph'>)>

# Adding Characters 

In [6]:
df=pd.read_csv('./database/Characters.csv',sep=",")
df['character_name'] = df['character_name'].str.replace(' ', '_')
df['constellation'] = df['constellation'].str.replace(' ', '_')
df['birthday'] = pd.to_datetime(df['birthday'], format='%d/%m/%Y')
df['release_date'] = pd.to_datetime(df['release_date'], format='%m/%d/%Y')
df['ascension_boss'] = df['ascension_boss'].str.replace(' ', '_')
df['ascension'] = df['ascension'].str.replace(' ', '_')
df.head()

,character_name,rarity,region,vision,weapon_type,constellation,birthday,ascension_boss,limited,ascension,role,gender,archon,release_date
0,Albedo,5,Mondstadt,Geo,Sword,Princeps_Cretaceus,2023-09-13,Basalt_Pillar,True,Geo_DMG_Bonus,Support,Male,False,2020-09-28
1,Alhaitham,5,Sumeru,Dendro,Sword,Vultur_Volans,2023-02-11,Pseudo-Stamens,True,Dendro_DMG_Bonus,DPS,Male,False,2023-01-18
2,Aloy,5,Mondstadt,Cryo,Bow,Nora_Fortis,2023-04-04,Crystalline_Bloom,True,Cryo_DMG_Bonus,Support,Female,False,2021-09-01
3,Amber,4,Mondstadt,Pyro,Bow,Lepus,2023-08-10,Everflame_Seed,False,ATK,Support,Female,False,2020-09-28
4,Arataki_Itto,5,Inazuma,Geo,Claymore,Taurus_Iracundus,2023-06-01,Riftborn_Regalia,True,CRIT_Rate,DPS,Male,False,2021-12-14


In [7]:
for index, row in df.iterrows():
    name = URIRef(gRes+row['character_name'])
    constellation = URIRef(gRes+row['constellation'])
    characterMaterial = URIRef(gRes+row['ascension_boss'])
    role = URIRef(gOnto+row['role'])
    
    g.add((name, RDF.type, URIRef(gOnto + "Character")))    
    g.add((name, RDF.type, OWL.NamedIndividual))
    g.add((name, FOAF.name, Literal(row['character_name'])))
    g.add((name, RDFS.label, Literal(row['character_name'], datatype = XSD.string)))
    
    g.add((constellation, RDF.type, URIRef(gOnto + "Constellation")))
    g.add((constellation, RDF.type, OWL.NamedIndividual))
    g.add((constellation, RDFS.label, Literal(row['constellation'], datatype = XSD.string)))
    
    g.add((characterMaterial, RDF.type, URIRef(gOnto + "Material")))
    g.add((characterMaterial, RDF.type, OWL.NamedIndividual))
    g.add((characterMaterial, RDFS.label, Literal(row['ascension_boss'], datatype = XSD.string)))

    g.add((role, RDF.type, URIRef(gOnto + "Role")))
    g.add((role, RDF.type, OWL.NamedIndividual))
    g.add((role, RDFS.label, Literal(row['role'], datatype = XSD.string)))
    
    g.add((name, URIRef(gOnto + "rarity"), Literal(row['rarity'], datatype = XSD.integer)))
    g.add((name, URIRef(gOnto + "residesAt"), URIRef(gRes + row['region'])))
    g.add((name, URIRef(gOnto + "elementAlignment"), URIRef(gRes + row['vision'])))
    g.add((name, URIRef(gOnto + "usesWeaponType"), URIRef(gOnto + row['weapon_type'])))
    g.add((name, URIRef(gOnto + "hasConstellation"), constellation))
    g.add((name, URIRef(gOnto + "birthday"), Literal(row['birthday'], datatype= XSD.dateTime)))
    g.add((name, URIRef(gOnto + "releaseDate"), Literal(row['release_date'], datatype= XSD.dateTime)))
    g.add((name, URIRef(gOnto + "usesCharacterUpgradeMaterial"), characterMaterial))
    g.add((name, URIRef(gOnto + "isLimitedCharacter"), Literal(row['limited'], datatype = XSD.boolean)))
    g.add((name, URIRef(gOnto + "isArchon"), Literal(row['archon'], datatype = XSD.boolean)))
    g.add((name, URIRef(gOnto + "hasAscensionPassive"), Literal(row['ascension'], datatype = XSD.string)))
    g.add((name, URIRef(gOnto + "hasRole"), role))
    g.add((name, URIRef(gOnto + "hasGender"), Literal(row['gender'], datatype = XSD.string)))
        

In [8]:
print(g.serialize(format='turtle'))

@prefix : <http://www.genshin.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix genshinOntology: <http://www.genshin.org/ontology/> .
@prefix genshinResource: <http://www.genshin.org/resource/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix swrl: <http://www.w3.org/2003/11/swrl#> .
@prefix swrla: <http://swrl.stanford.edu/ontologies/3.3/swrla.owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

swrla:isRuleEnabled a owl:AnnotationProperty .

genshinOntology: a owl:Ontology ;
    owl:versionIRI <http://www.genshin.org/ontology/1.0.0/> .

genshinOntology:AdvancedReaction a owl:Class ;
    rdfs:comment "Those specific reactions where more than 2 elements can participate in" ;
    rdfs:subClassOf genshinOntology:Reaction ;
    owl:equivalentClass [ a owl:Restriction ;
            owl:minQualifiedCardinality "3"^^xsd:nonNegativeInteger ;
         

In [9]:
g.serialize(destination="afterAddingCharacters.rdf", format="application/rdf+xml")

<Graph identifier=Nf1cc6f9c6da0490896c863e1456403dc (<class 'rdflib.graph.Graph'>)>

# Adding Enemies (Listen Thorfinn, my son. You have no enemies.)

In [10]:
df=pd.read_csv('./database/Enemies.csv',sep=",")
df['Name'] = df['Name'].str.replace(' ', '_')
df['DropsMaterial'] = df['DropsMaterial'].str.replace(' ', '_')
df['EnemyType'] = df['EnemyType'].str.replace(' ', '')
df.head()

,Name,Element,WeakAgainst,EnemyType,DropsMaterial
0,Hilichurl,Anemo,Electro,Normal,Firm_Arrowhead
1,Anemo_Samachurl,Anemo,Electro,Elite,Divining_Scroll
2,Hydro_Samachurl,Hydro,Pyro,Elite,Divining_Scroll
3,Cryo_Samachurl,Cryo,Pyro,Elite,Divining_Scroll
4,Mitachurl,Anemo,Electro,Elite,Firm_Arrowhead


In [11]:
for index, row in df.iterrows():
    name = URIRef(gRes+row['Name'])
    material = URIRef(gRes+row['DropsMaterial'])
    
    g.add((name, RDF.type, URIRef(gOnto + "Enemy")))    
    g.add((name, RDF.type, OWL.NamedIndividual))
    g.add((name, FOAF.name, Literal(row['Name'])))
    g.add((name, RDFS.label, Literal(row['Name'], datatype = XSD.string)))
    
    g.add((material, RDF.type, URIRef(gOnto + "Material")))
    g.add((material, RDF.type, OWL.NamedIndividual))
    g.add((material, RDFS.label, Literal(row['DropsMaterial'], datatype = XSD.string)))
    
    g.add((name, URIRef(gOnto + "elementAlignment"), URIRef(gRes + row['Element'])))
    g.add((name, URIRef(gOnto + "enemyWeakAgainst"), URIRef(gRes + row['WeakAgainst'])))
    g.add((name, URIRef(gOnto + "isEnemyType"), URIRef(gOnto + row['EnemyType'])))
    g.add((name, URIRef(gOnto + "dropsMaterial"), material))        

In [12]:
print(g.serialize(format='turtle'))

@prefix : <http://www.genshin.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix genshinOntology: <http://www.genshin.org/ontology/> .
@prefix genshinResource: <http://www.genshin.org/resource/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix swrl: <http://www.w3.org/2003/11/swrl#> .
@prefix swrla: <http://swrl.stanford.edu/ontologies/3.3/swrla.owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

swrla:isRuleEnabled a owl:AnnotationProperty .

genshinOntology: a owl:Ontology ;
    owl:versionIRI <http://www.genshin.org/ontology/1.0.0/> .

genshinOntology:AdvancedReaction a owl:Class ;
    rdfs:comment "Those specific reactions where more than 2 elements can participate in" ;
    rdfs:subClassOf genshinOntology:Reaction ;
    owl:equivalentClass [ a owl:Restriction ;
            owl:minQualifiedCardinality "3"^^xsd:nonNegativeInteger ;
         

In [13]:
g.serialize(destination="afterAddingEnemies.rdf", format="application/rdf+xml")

<Graph identifier=Nf1cc6f9c6da0490896c863e1456403dc (<class 'rdflib.graph.Graph'>)>

# Adding Weapons

In [14]:
df=pd.read_csv('./database/Weapons.csv',sep=",")
df['Name'] = df['Name'].str.replace(' ', '_')
df['UpgradeMaterial'] = df['UpgradeMaterial'].str.replace(' ', '_')
df.head()

,Name,Type,Rarity,UpgradeMaterial
0,Wolf's_Gravestone,Claymore,5,Tusk_of_the_Wolf
1,Skyward_Harp,Bow,5,Luminous_Sands_from_Guyun
2,Primordial_Jade_Winged-Spear,Polearm,5,Jade_Wing
3,Aquila_Favonia,Sword,5,Tusk_of_the_Wolf
4,Lost_Prayer_to_the_Sacred_Winds,Catalyst,5,Tile_of_Decarabian's_Tower


In [15]:
for index, row in df.iterrows():
    name = URIRef(gRes+row['Name'])
    weaponMaterial = URIRef(gRes+row['UpgradeMaterial'])
    
    g.add((name, RDF.type, URIRef(gOnto + "Weapon")))    
    g.add((name, RDF.type, OWL.NamedIndividual))
    g.add((name, FOAF.name, Literal(row['Name'])))
    g.add((name, RDFS.label, Literal(row['Name'], datatype = XSD.string)))
    
    g.add((weaponMaterial, RDF.type, URIRef(gOnto + "Material")))
    g.add((weaponMaterial, RDF.type, OWL.NamedIndividual))
    g.add((weaponMaterial, FOAF.name, Literal(row['UpgradeMaterial'])))
    g.add((weaponMaterial, RDFS.label, Literal(row['UpgradeMaterial'], datatype = XSD.string)))
    
    g.add((name, URIRef(gOnto + "isWeaponType"), URIRef(gOnto + row['Type'])))
    g.add((name, URIRef(gOnto + "rarity"), Literal(row['Rarity'], datatype = XSD.integer)))
    g.add((name, URIRef(gOnto + "usesWeaponUpgradeMaterial"), weaponMaterial))
    

In [16]:
print(g.serialize(format='turtle'))

@prefix : <http://www.genshin.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix genshinOntology: <http://www.genshin.org/ontology/> .
@prefix genshinResource: <http://www.genshin.org/resource/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix swrl: <http://www.w3.org/2003/11/swrl#> .
@prefix swrla: <http://swrl.stanford.edu/ontologies/3.3/swrla.owl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

swrla:isRuleEnabled a owl:AnnotationProperty .

genshinOntology: a owl:Ontology ;
    owl:versionIRI <http://www.genshin.org/ontology/1.0.0/> .

genshinOntology:AdvancedReaction a owl:Class ;
    rdfs:comment "Those specific reactions where more than 2 elements can participate in" ;
    rdfs:subClassOf genshinOntology:Reaction ;
    owl:equivalentClass [ a owl:Restriction ;
            owl:minQualifiedCardinality "3"^^xsd:nonNegativeInteger ;
         

In [17]:
g.serialize(destination="afterAddingWeapons.rdf", format="application/rdf+xml")

<Graph identifier=Nf1cc6f9c6da0490896c863e1456403dc (<class 'rdflib.graph.Graph'>)>

# Linking with Wikidata

In [18]:
# Keqing
character = URIRef(gRes + 'Keqing')
wikidataLink = URIRef(wd + 'Q107287676')
g.add((character, OWL.sameAs, wikidataLink))

# Yun Jin
character = URIRef(gRes + 'Yun_Jin')
wikidataLink = URIRef(wd + 'Q110653384')
g.add((character, OWL.sameAs, wikidataLink))

# Shenhe
character = URIRef(gRes + 'Shenhe')
wikidataLink = URIRef(wd + 'Q110677319')
g.add((character, OWL.sameAs, wikidataLink))

# Ganyu
character = URIRef(gRes + 'Ganyu')
wikidataLink = URIRef(wd + 'Q107618618')
g.add((character, OWL.sameAs, wikidataLink))

# Diluc
character = URIRef(gRes + 'Diluc')
wikidataLink = URIRef(wd + 'Q107263978')
g.add((character, OWL.sameAs, wikidataLink))

# Hu Tao
character = URIRef(gRes + 'Barbara')
wikidataLink = URIRef(wd + 'Q110911467')
g.add((character, OWL.sameAs, wikidataLink))

# Yanfei
character = URIRef(gRes + 'Yanfei')
wikidataLink = URIRef(wd + 'Q110965539')
g.add((character, OWL.sameAs, wikidataLink))

# Amber
character = URIRef(gRes + 'Amber')
wikidataLink = URIRef(wd + 'Q111008775')
g.add((character, OWL.sameAs, wikidataLink))

# Xiangling
character = URIRef(gRes + 'Xiangling')
wikidataLink = URIRef(wd + 'Q111018815')
g.add((character, OWL.sameAs, wikidataLink))

# Noelle
character = URIRef(gRes + 'Noelle')
wikidataLink = URIRef(wd + 'Q107291860')
g.add((character, OWL.sameAs, wikidataLink))

# Eula
character = URIRef(gRes + 'Eula')
wikidataLink = URIRef(wd + 'Q111079705')
g.add((character, OWL.sameAs, wikidataLink))

# Lisa
character = URIRef(gRes + 'Lisa')
wikidataLink = URIRef(wd + 'Q111133790')
g.add((character, OWL.sameAs, wikidataLink))

# Ningguang
character = URIRef(gRes + 'Ningguang')
wikidataLink = URIRef(wd + 'Q111180134')
g.add((character, OWL.sameAs, wikidataLink))

# Yae Miko
character = URIRef(gRes + 'Yae_Miko')
wikidataLink = URIRef(wd + 'Q111631165')
g.add((character, OWL.sameAs, wikidataLink))

# Kazuha
character = URIRef(gRes + 'Kaedehara_Kazuha')
wikidataLink = URIRef(wd + 'Q111684393')
g.add((character, OWL.sameAs, wikidataLink))

# Raiden
character = URIRef(gRes + 'Raiden_Shogun')
wikidataLink = URIRef(wd + 'Q111655115')
g.add((character, OWL.sameAs, wikidataLink))

# Sucrose
character = URIRef(gRes + 'Sucrose')
wikidataLink = URIRef(wd + 'Q112220215')
g.add((character, OWL.sameAs, wikidataLink))

# Chongyun
character = URIRef(gRes + 'Chongyun')
wikidataLink = URIRef(wd + 'Q107291862')
g.add((character, OWL.sameAs, wikidataLink))

# Ayaka
character = URIRef(gRes + 'Kamisato_Ayaka')
wikidataLink = URIRef(wd + 'Q107618679')
g.add((character, OWL.sameAs, wikidataLink))

# Yoimiya
character = URIRef(gRes + 'Yoimiya')
wikidataLink = URIRef(wd + 'Q113493715')
g.add((character, OWL.sameAs, wikidataLink))

# Fischl
character = URIRef(gRes + 'Fischl')
wikidataLink = URIRef(wd + 'Q113499833')
g.add((character, OWL.sameAs, wikidataLink))

# Thoma
character = URIRef(gRes + 'Thoma')
wikidataLink = URIRef(wd + 'Q113668570')
g.add((character, OWL.sameAs, wikidataLink))

# Nahida
character = URIRef(gRes + 'Nahida')
wikidataLink = URIRef(wd + 'Q115232300')
g.add((character, OWL.sameAs, wikidataLink))

# Xiao
character = URIRef(gRes + 'Xiao')
wikidataLink = URIRef(wd + 'Q107263982')
g.add((character, OWL.sameAs, wikidataLink))

# Qiqi
character = URIRef(gRes + 'Qiqi')
wikidataLink = URIRef(wd + 'Q107287683')
g.add((character, OWL.sameAs, wikidataLink))

# Yelan
character = URIRef(gRes + 'Yelan')
wikidataLink = URIRef(wd + 'Q117195714')
g.add((character, OWL.sameAs, wikidataLink))

# Collei
character = URIRef(gRes + 'Collei')
wikidataLink = URIRef(wd + 'Q118153805')
g.add((character, OWL.sameAs, wikidataLink))

# Kokomi
character = URIRef(gRes + 'Sangonomiya_Kokomi')
wikidataLink = URIRef(wd + 'Q119998675')
g.add((character, OWL.sameAs, wikidataLink))

# Heizou
character = URIRef(gRes + 'Shikanoin_Heizou')
wikidataLink = URIRef(wd + 'Q120994155')
g.add((character, OWL.sameAs, wikidataLink))

# Candace
character = URIRef(gRes + 'Candace')
wikidataLink = URIRef(wd + 'Q121919991')
g.add((character, OWL.sameAs, wikidataLink))


<Graph identifier=Nf1cc6f9c6da0490896c863e1456403dc (<class 'rdflib.graph.Graph'>)>

In [19]:
print(g.serialize(format='turtle'))

@prefix : <http://www.genshin.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix genshinOntology: <http://www.genshin.org/ontology/> .
@prefix genshinResource: <http://www.genshin.org/resource/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix swrl: <http://www.w3.org/2003/11/swrl#> .
@prefix swrla: <http://swrl.stanford.edu/ontologies/3.3/swrla.owl#> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

swrla:isRuleEnabled a owl:AnnotationProperty .

genshinOntology: a owl:Ontology ;
    owl:versionIRI <http://www.genshin.org/ontology/1.0.0/> .

genshinOntology:AdvancedReaction a owl:Class ;
    rdfs:comment "Those specific reactions where more than 2 elements can participate in" ;
    rdfs:subClassOf genshinOntology:Reaction ;
    owl:equivalentClass [ a owl:Restriction ;
            owl:minQualifiedCar

In [20]:
g.serialize(destination="afterLinking.rdf", format="application/rdf+xml")

<Graph identifier=Nf1cc6f9c6da0490896c863e1456403dc (<class 'rdflib.graph.Graph'>)>

# Final Save

In [21]:
g.serialize(destination="afterAddingEverything.rdf", format="application/rdf+xml")

<Graph identifier=Nf1cc6f9c6da0490896c863e1456403dc (<class 'rdflib.graph.Graph'>)>